In [24]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import resample

import time
import random

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, precision_score, accuracy_score,f1_score, recall_score
from sklearn.linear_model import SGDClassifier
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import PrecisionRecallCurve
from sklearn.model_selection import cross_val_predict, cross_val_score


from sklearn.feature_selection import RFE
from boruta import BorutaPy

In [27]:
random_seed=42

In [17]:
df_train = pd.read_csv('train.csv')


In [18]:
df_train.head()

,Unnamed: 0,acc_now_delinq,annual_inc,joint_app,chargeoff_within_12_mths,collection_recovery_fee,collections_12_mths_ex_med,delinq_2yrs,delinq_amnt,dti,...,tax_liens.1,term.1,total_acc.1,total_pymnt.1,total_pymnt_inv.1,total_rec_int.1,total_rec_late_fee.1,total_rec_prncp.1,earliest_cr_line_date.1,loan_status
0,52309,0.0,0.017296,0.0,0.0,0.0,0.0,0.0,0.0,0.042479,...,0.0,1.0,0.305085,0.556515,0.557348,0.299021,1.000981e-11,0.6500,0.579996,0
1,38014,0.0,0.008719,0.0,0.0,0.0,0.0,0.0,0.0,0.017815,...,0.0,1.0,0.093220,0.732477,0.733149,0.595846,1.000981e-11,0.7275,0.738631,0
2,49445,0.0,0.008719,0.0,0.0,0.0,0.0,0.0,0.0,0.031740,...,0.0,0.0,0.398305,0.138887,0.140101,0.056731,1.000981e-11,0.1750,0.713353,0
3,25480,0.0,0.012007,0.0,0.0,0.0,0.0,0.0,0.0,0.021917,...,0.0,0.0,0.237288,0.149915,0.151119,0.027469,1.000981e-11,0.2100,0.809340,0
4,22883,0.0,0.008576,0.0,0.0,0.0,0.0,0.0,0.0,0.039267,...,0.0,1.0,0.381356,0.488459,0.489354,0.065163,1.000981e-11,0.6950,0.693332,0


In [19]:
df_train.drop('Unnamed: 0',axis=1,inplace=True)

In [20]:
X_train=df_train.drop('loan_status',axis=1)
y_train=df_train.loan_status

In [21]:
X_train.shape 

(34563, 1652)

1652 variables, far too many to be implemented. Can this be reduced?

Take best model from imbalance strategy comparison (logistic regression with oversampling)

In [22]:
df_majority = df_train[df_train['loan_status']==0]
df_minority = df_train[df_train['loan_status']==1]

In [28]:
df_minority_oversampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=random_seed)
df_os = pd.concat([df_minority_oversampled, df_majority])

In [30]:
y_train_os = df_os['loan_status']
X_train_os = df_os.drop('loan_status', axis=1)

In [32]:
#best model
model = LogisticRegression(C=10, class_weight = 'balanced', max_iter = 100)

In [33]:
model.fit(X_train_os,y_train_os)

LogisticRegression(C=10, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
importance =abs( model.coef_[0])
importance

array([ 0.43554461,  0.36690862,  0.73423179, ...,  6.77633025,
       42.37786181,  0.09244833])

In [35]:
feature_name = X_train_sm.columns.values
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
summary_table['Coefficients'] = np.transpose(model.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', model.intercept_[0]] # intercept is not in column names
summary_table = summary_table.sort_index()
summary_table

NameError: name 'X_train_sm' is not defined

In [ ]:
boruta_selector = BorutaPy(forest, n_estimators = 'auto', random_state = 0,perc=90,verbose=2)
boruta_selector.fit(np.array(X_train), np.array(y_train))